In [ ]:
import uuid
import psycopg2

### Connecting to database

In [ ]:
# Connecting to database
conn = psycopg2.connect(
    host = 'localhost',
    database = 'AdventureWorks',
    user = 'postgres',
    password = 'postgres')

In [ ]:
# Creating a cursor
cur = conn.cursor()

In [ ]:
print('PostgreSQL database version:')
cur.execute('SELECT version()')
print(cur.fetchone())

### Marking a simple transaction

In [ ]:
# Executing a first statement
cur.execute('SELECT COUNT(*) FROM sales.salesorderheader;')
print(cur.fetchone())

# Executing a second statement
cur.execute('SELECT COUNT(*) FROM sales.salesorderdetail;')
print(cur.fetchone())

# Commit the transaction
conn.commit()

### Marking a more complex transaction

In [ ]:
# Adding a new category
cur.execute('''
    INSERT INTO production.productcategory
    (productcategoryid, name, rowguid, modifieddate)
    VALUES
    (%s, %s, %s, NOW())
    RETURNING productcategoryid
''', (5, 'Test category', str(uuid.uuid4())))

productcategoryid = cur.fetchone()[0]

# Adding a new subcategory
cur.execute('''
    INSERT INTO production.productsubcategory
    (productsubcategoryid, productcategoryid, name, rowguid, modifieddate)
    VALUES
    (%s, %s, %s, %s, NOW())
    RETURNING productsubcategoryid
''', (38, productcategoryid, 'Test subcategory', str(uuid.uuid4())))

productsubcategoryid = cur.fetchone()[0]

# Adding a new product
cur.execute('''
    INSERT INTO production.product
    (productid, name, productnumber, makeflag, finishedgoodsflag, safetystocklevel, reorderpoint, standardcost, listprice, daystomanufacture, productsubcategoryid, sellstartdate, rowguid, modifieddate)
    VALUES
    (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, NOW(), %s, NOW())
''', (5000, 'Test product 1', 'TEST-0001', False, False, 1, 1, 80, 100, 7, productsubcategoryid, str(uuid.uuid4())))

conn.commit()

### A transaction that will fail

In [ ]:
try:
    # Adding a new subcategory
    cur.execute('''
        INSERT INTO production.productsubcategory
        (productsubcategoryid, productcategoryid, name, rowguid, modifieddate)
        VALUES
        (%s, %s, %s, %s, NOW())
        RETURNING productsubcategoryid
    ''', (39, productcategoryid, 'Test subcategory 2', str(uuid.uuid4())))

    productsubcategoryid = cur.fetchone()[0]

    # Adding a new product
    cur.execute('''
        INSERT INTO production.product
        (productid, name, productnumber, makeflag, finishedgoodsflag, safetystocklevel, reorderpoint, standardcost, listprice, daystomanufacture, productsubcategoryid, sellstartdate, rowguid, modifieddate)
        VALUES
        (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, NOW(), %s, NOW())
    ''', (5000, 'Test product 2', 'TEST-0002', False, False, 1, 1, 80, 100, 7, productsubcategoryid, str(uuid.uuid4())))

    conn.commit()
except Exception as e:
    print ('Oops! An exception has occured:', type(e))
    print (e)
    
    print('Rollbacked transaction')
    conn.rollback()

### Develop a transaction for creating a new sales order

Recuerde que:
- Una orden está compuesta por un encabezado (salesorderheader) y uno o varios detalles o items (salesorderdetail).
- Para la inserción de un encabezado se debe validar previamente que como mínimo exista el cliente (customer), la dirección de facturación (billtoaddress), la dirección de envío (shiptoaddress) y el método de envío (shipmethod).
- Para la inserción de un detalle, además de validar previamente que exista el encabezado, se debe validar que exista el producto (product) y si se está aplicando alguna oferta especial (specialoffer).

Adicionalmente, para que la transacción sea exitosa, asegurese de que la suma de los precios de cada detalle ((unitprice - unitpricediscount) * orderqty) coincidan con el subtotal establecido en el encabezado de la orden.

### Closing connection

In [ ]:
# Closing the cursor and connection
cur.close()
conn.close()